In [1]:
import pandas as pd
import numpy as np

In [150]:
# Definir o tamanho do dataframe de treino e teste
num_samples_train = 1000
num_samples_test = 500

# Gerar dados fictícios para as variáveis explicativas sem missings
var1_train = np.random.randint(0, 10000, size=num_samples_train)
var2_train = np.random.randint(0, 10000, size=num_samples_train)
var3_train = np.random.randint(0, 10000, size=num_samples_train)

var1_test = np.random.randint(0, 10000, size=num_samples_test)
var2_test = np.random.randint(0, 10000, size=num_samples_test)
var3_test = np.random.randint(0, 10000, size=num_samples_test)

# Gerar os rótulos da variável target (RESP)
target_train = np.random.choice([0, 1], size=num_samples_train)
target_test = np.random.choice([0, 1], size=num_samples_test)

# Criar o dataframe de treino
train = pd.DataFrame({'VAR1': var1_train, 'VAR2': var2_train, 'VAR3': var3_train, 'RESP': target_train})

# Criar o dataframe de teste
test = pd.DataFrame({'VAR1': var1_test, 'VAR2': var2_test, 'VAR3': var3_test, 'RESP': target_test})

# Substituir 20% dos valores de cada variável explicativa (exceto target) por NaN (missing)
train.loc[:, 'VAR1'].iloc[np.random.choice(num_samples_train, int(num_samples_train * 0.2), replace=False)] = np.nan
train.loc[:, 'VAR2'].iloc[np.random.choice(num_samples_train, int(num_samples_train * 0.2), replace=False)] = np.nan
train.loc[:, 'VAR3'].iloc[np.random.choice(num_samples_train, int(num_samples_train * 0.2), replace=False)] = np.nan

test.loc[:, 'VAR1'].iloc[np.random.choice(num_samples_test, int(num_samples_test * 0.2), replace=False)] = np.nan
test.loc[:, 'VAR2'].iloc[np.random.choice(num_samples_test, int(num_samples_test * 0.2), replace=False)] = np.nan
test.loc[:, 'VAR3'].iloc[np.random.choice(num_samples_test, int(num_samples_test * 0.2), replace=False)] = np.nan


<ipython-input-150-5b22d898a2da>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.loc[:, 'VAR2'].iloc[np.random.choice(num_samples_train, int(num_samples_train * 0.2), replace=False)] = np.nan
<ipython-input-150-5b22d898a2da>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.loc[:, 'VAR3'].iloc[np.random.choice(num_samples_train, int(num_samples_train * 0.2), replace=False)] = np.nan
<ipython-input-150-5b22d898a2da>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [162]:
train['C_VAR1'], bins = pd.cut(train['VAR1'], 5, retbins=True, duplicates='drop', include_lowest=False)

In [163]:
train.groupby('C_VAR1')['RESP'].mean().sort_values()

C_VAR1
(4000.2, 5995.8]    0.493827
(5995.8, 7991.4]    0.532051
(2004.6, 4000.2]    0.533784
(-0.978, 2004.6]    0.538012
(7991.4, 9987.0]    0.564417
Name: RESP, dtype: float64

In [9]:
sorted(bins)

[31.043, 2032.4, 4023.8, 6015.200000000001, 8006.6, 9998.0]

In [158]:
def categoriza_cont(train, test, target_col, feature_col, num_bins, log=False):
    # Criar cópia do dataframe original para evitar alterações indesejadas
    df = train.copy()

    # Criar bins usando a função cut do pandas
    if log:
      df['bins'], interval = pd.cut(np.log(df[feature_col]+1), num_bins, retbins=True, duplicates='drop', include_lowest=False)
    else:
      df['bins'], interval = pd.cut(df[feature_col], num_bins, retbins=True, duplicates='drop', include_lowest=False)

    # Calcular a bad rate em cada domínio
    bad_rate = df.groupby('bins')[target_col].mean()

    # Calcular a bad rate em missings
    # missing = df[df[feature_col].isna()][target_col].mean()

    #Juntar e Ordenar
    # bad_rate = bad_rate.append(pd.Series({np.nan: missing}))
    sorted_bins = bad_rate.sort_values()  

    
    # Mapear as categorias para os índices ordenados
    bin_mapping = {bin_val: i for i, bin_val in enumerate(sorted_bins.index)}
    df[f'C_{feature_col}'] = df['bins'].map(bin_mapping).astype(float)
    df.loc[df[feature_col].isna(), f'C_{feature_col}'] = -1
    # df.drop('bins',axis=1, inplace=True)

    # Aplicação das mesmas regras na base de teste
    if log:
      test[f'C_{feature_col}'] = pd.cut(np.log(test[feature_col]+1), bins=interval, labels=False)
    else:
      test[f'C_{feature_col}'] = pd.cut(test[feature_col], bins=interval, labels=False)

    test[f'C_{feature_col}'] = test[f'C_{feature_col}'].map(bin_mapping).astype(float)
    test.loc[test[feature_col].isna(), f'C_{feature_col}'] = -1

    return df, test, bin_mapping


In [159]:
train, test, bin1 = categoriza_cont(train,test,'RESP','VAR1', 5)
train, test, bin2 = categoriza_cont(train,test,'RESP','VAR2', 5)
train, test, bin3 = categoriza_cont(train,test,'RESP','VAR3', 10,log=True)

In [160]:
bin3

{Interval(2.558, 3.229, closed='right'): 0,
 Interval(4.558, 5.223, closed='right'): 1,
 Interval(6.551, 7.216, closed='right'): 2,
 Interval(3.894, 4.558, closed='right'): 3,
 Interval(8.545, 9.209, closed='right'): 4,
 Interval(5.887, 6.551, closed='right'): 5,
 Interval(7.88, 8.545, closed='right'): 6,
 Interval(5.223, 5.887, closed='right'): 7,
 Interval(7.216, 7.88, closed='right'): 8,
 Interval(3.229, 3.894, closed='right'): 9}

In [161]:
test['VAR3LOG'] = np.log(test['VAR3']+1)
test[test['C_VAR3'].isna()]

,VAR1,VAR2,VAR3,RESP,C_VAR1,C_VAR2,C_VAR3,VAR3LOG
126,1365.0,3753.0,30.0,1,3.0,2.0,NaN,3.433987
246,6154.0,9078.0,16.0,1,3.0,2.0,NaN,2.833213
253,7835.0,3269.0,9991.0,1,3.0,2.0,NaN,9.209540
262,3484.0,9221.0,23.0,0,3.0,2.0,NaN,3.178054


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


0     -inf
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
495    0.0
496    NaN
497    0.0
498    0.0
499   -inf
Name: C_VAR3, Length: 500, dtype: float64

In [ ]:
def categorizar_variaveis_continuas(base_treino, base_teste, variaveis, variavel_alvo, bins):
    for variavel in variaveis:
        # Categorização da variável contínua usando pd.cut
        base_treino[f'{variavel}_categorizada'], intervalos = pd.cut(base_treino[variavel], bins, labels=False, retbins=True)

        # Criação da tabela de frequências e cálculo da Bad Rate por categoria
        tabela_frequencias = pd.crosstab(base_treino[f'{variavel}_categorizada'], base_treino[variavel_alvo])
        tabela_frequencias['bad_rate'] = tabela_frequencias[1] / tabela_frequencias.sum(axis=1)

        # Cálculo da Bad Rate para os valores missing
        bad_rate_missing = base_treino[variavel_alvo].isnull().sum() / len(base_treino)

        # Adição da Bad Rate dos valores missing à tabela de frequências
        tabela_frequencias.loc[np.nan, 'bad_rate'] = bad_rate_missing

        # Ordenação das categorias pela Bad Rate
        categorias_ordenadas = tabela_frequencias.sort_values('bad_rate').index.tolist()

        # Atribuição de rótulos em ordem crescente
        rotulos = range(len(categorias_ordenadas))
        rotulos_dict = {categoria: rotulo for categoria, rotulo in zip(categorias_ordenadas, rotulos)}

        # Aplicação das regras na base de treino
        base_treino[f'{variavel}_categorizada'] = base_treino[f'{variavel}_categorizada'].map(rotulos_dict)

        # Aplicação das mesmas regras na base de teste
        base_teste[f'{variavel}_categorizada'] = pd.cut(base_teste[variavel], bins=intervalos, labels=False)
        base_teste[f'{variavel}_categorizada'] = base_teste[f'{variavel}_categorizada'].map(rotulos_dict)

    return base_treino, base_teste

In [70]:
train[train['VAR1'].isna()]

,VAR1,VAR2,VAR3,RESP
3,NaN,8921.0,8851.0,0
9,NaN,6560.0,8570.0,0
20,NaN,1346.0,2141.0,0
21,NaN,124.0,NaN,0
23,NaN,9379.0,3171.0,1
...,...,...,...,...
986,NaN,NaN,4873.0,0
988,NaN,224.0,5102.0,1
991,NaN,8574.0,5407.0,1
997,NaN,4944.0,793.0,0


In [ ]:
def categoriza_cont()